In [1]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures
using Colors
using Random

# Why Preorders?

We simplify the question of prerequisites 

1. To make the question more manageable to think about
2. To showcase preorders as a way of organizing information

With preorders, we only deal with prerequisites that are other classes. Prerequisites with that include **or** are not too hard to generalize to, but other types of prerequisites increase complexity tremendously. For example, prerequisites of class standing can go in both directions, causing classes no longer to be available just by spending too long in school. Furthermore, some majors require letter grades higher than the standard C in a course.

In short, we avoid rare and mostly needless complications by working with a preorder structure.

# The Preorder

In [2]:
# taken from programming exercise 1
struct Preorder
  carrier:: FinSet
  relation :: SortedSet{Pair{Int,Int}}
  """Construct valid preorder by taking reflexive/transitive closure"""
  function Preorder(carrier:: FinSet, rel :: Vector{Pair{Int,Int}})
    for (a,b) ∈ rel
      a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
    end
    relation = SortedSet(rel)
    for (a, b, c) in Iterators.product(carrier, carrier, carrier)
      if ((a => b) ∈ relation) && ((b => c) ∈ relation)
        push!(relation, a => c) # enforces relation is transitive
      end
    end
    for i in carrier
      push!(relation, i => i) # enforces that relation is reflexive
    end
    return new(carrier, relation)
  end
end

![title](Images/PreorderDefinition.png)

In the catalog, classes are presented as having prerequisites. We take a different view: with preorders, we store the classes in **carrier** and then define the prerequisites in a separate collection **relation** by adding pairs (a,b) when a is a prerequisite of b. Then we turn our data into a preorder by taking the reflexive and transitive closure.

But notice that we do not have our information stored directly in preorders. Instead, we have our carrier as a FinSet and our relations. 

The fundamental fact is that any relabelling of the carrier gives us a preorder with the exact same shape. That is, preorders are not affected by the names of the objects that make up the carrier; the information is in how the objects relate. So we represent our finite preorder of classes using a FinSet and simly keep a map between class names and number to give meaning to the preorder. 

This practice of separating the structure and using a map to give meaning to the objects appears again as the far less trivial practice of filling a database schema with a set-valued functor.

In [3]:
# function to parse major requirement files
function load_major(path::String, taken_classes::Set{String}=Set{String}())

    file = open(path, "r")

    num_to_class = Dict()
    class_to_num = Dict()

    relations = Vector{Pair{Int, Int}}()

    num_classes = 0
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                if !in(name, taken_classes)
                    num_classes+=1
                    num_to_class[num_classes] = string(name)
                    class_to_num[string(name)] = num_classes
                end

                if length(sp) >= 2
                    reqs_str = strip(sp[2], ['[', ']', ' '])
                    reqs = split(reqs_str, ",")
                    
                    for req in reqs
                        req = strip(req, ' ')
                        if length(req) > 0
                            if !in(req, taken_classes) && in(name, taken_classes)
                                throw("At least one class taken has missing prereqs!")
                            end
                            if !in(req, taken_classes)
                                push!(relations, class_to_num[req] => class_to_num[name])
                            end
                        end
                    end
                end
            end
        end
    end

    close(file)


    return (Preorder(FinSet(num_classes),relations), num_to_class)
end

load_major (generic function with 2 methods)

In [4]:
# load full degree requirements into preorder
cs, cs_mappings = load_major("reqs/CS.reqs")

(Preorder(FinSet(35), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 9, 1 => 10, 1 => 11  …  28 => 29, 28 => 30, 29 => 29, 29 => 30, 30 => 30, 31 => 31, 32 => 32, 33 => 33, 34 => 34, 35 => 35],
Base.Order.ForwardOrdering())), Dict{Any, Any}(5 => "STA3032", 16 => "COP4600", 20 => "CIS4930-1", 35 => "HumOrSocOrPhy1", 12 => "COP3530", 24 => "CIS4930-5", 28 => "Interdisciplinary-3", 8 => "ENC3246", 17 => "COP4020", 30 => "Interdisciplinary-5"…))

In [5]:
# load degree requirements without some classes that were already taken into preorder
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)

(Preorder(FinSet(33), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 6, 1 => 10, 1 => 12, 1 => 14, 1 => 15, 1 => 16, 1 => 17  …  26 => 27, 26 => 28, 27 => 27, 27 => 28, 28 => 28, 29 => 29, 30 => 30, 31 => 31, 32 => 32, 33 => 33],
Base.Order.ForwardOrdering())), Dict{Any, Any}(5 => "PHY2048/PHY2048L", 16 => "COP4533", 20 => "CIS4930-3", 12 => "CEN3031", 24 => "Interdisciplinary-1", 28 => "Interdisciplinary-5", 8 => "COP3503", 17 => "CNT4007", 30 => "Humanities2", 1 => "MAC2312"…))

In [6]:
# try to load degree requirements with invalid classes taken
taken_classes = Set{String}(["COP4600"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)

LoadError: "At least one class taken has missing prereqs!"

# Monotone Maps and Valid Schedules

![title](Images/MonotoneMapDefinition.png)

The important thing to note is that the spirit of monotinicity is the preservation of order in the direction of the map.

Now we do not want to take prerequisites for a class in the same semester as the class, so we add strictness: we say that f is strictly monotone if $a < b \implies f(a) < f(b)$, where  $a < b$ if $a \leq b, and  ,a \ne b$
            
and model our semesters with a **Total Order**

![title](Images/TotalOrderDefinition.png)

Finite total orders of size n look like the first n elements of the natural number line, which is exactly the struture we are looking for in representing the progression of semesters as a preorder.

![title](Images/NaturalNumbers.png)

We say

1. A class is valid to take in a semester if all of its prerequisites have been fulfilled
2. A semester is valid if every class in the semester is valid
3. A schedule is valid every semester is valid to take given that all classes in previous semesters have been fulfilled
 
We can restate each of these definitions as respecting the order of prerequisites and it becomes clear that a Strict Monotone Map from our class preorder to a linear order gives a valid schedule.

# Generating a schedule

Our method is to build a valid schedule is to go semester by semester, filling each with valid classes. We do this by taking a number of valid classes from the "bottom" of the preorder to fill a semester and then discarding those classes as fulfilled for the next semester.

In [7]:
# finds elements with no edges in or out
function find_one_offs(preorder::Preorder)
    one_offs = Vector()
    d = Dict()
    for class in collect(preorder.carrier)
        d[class] = true
    end
    for (a,b) in preorder.relation
        if a != b
            d[a] = false
            d[b] = false
        end
    end
    for (class, is_one_off) in d
        if is_one_off
            push!(one_offs, class)
        end
    end
    return Set(one_offs)
end

function generate_schedule(preorder::Preorder, num_semesters::Int=8, seed::Int=0)
    Random.seed!(seed)

    classes = collect(preorder.carrier)
    reqs = preorder.relation

    full_schedule = Vector()

    one_off_classes = find_one_offs(preorder)


    # build dictionary of in counts like in topological sort
    in_counts = Dict()
    for class in classes
        # ignore one off classes for now
        if !(class in one_off_classes)
            in_counts[class] = 0
            for (a,b) in preorder.relation
                if class == b && a != b
                    in_counts[class]+=1
                end
            end
        end
    end


    # schedule classes with prereqs
    for index in 1:num_semesters
        semester_schedule = Vector()

        available_classes = Set(collect(in_counts))

        # find bottoms
        # marking them for deletion
        for (class, in_count) in shuffle!(collect(available_classes))
            if in_count == 0 && length(semester_schedule) < ceil(length(preorder.carrier)/num_semesters)
                push!(semester_schedule, class)
            end
        end

        # delete taken classes
        for class in semester_schedule
            for (a,b) in preorder.relation
                if class == a
                    in_counts[b]-=1
                end
            end
            delete!(in_counts, class)
        end

        push!(full_schedule, semester_schedule)
    end


    # schedule classes with no prereqs
    # prioritizing even credit distribution
    while length(one_off_classes) > 0
        
        # find semester with min number of credits
        min_index = 1
        for (index, semester) in enumerate(full_schedule)
            if length(full_schedule[index]) < length(full_schedule[min_index])
                min_index = index
            end 
        end
        
        class_to_add = collect(one_off_classes)[1]
        push!(full_schedule[min_index], class_to_add)
        delete!(one_off_classes, class_to_add)
    end

    # check that all classes were fit into the schedule
    if length(one_off_classes) > 0 || length(collect(in_counts)) > 0
        throw("Cannot generate schedule!")
    end

    return full_schedule
end

generate_schedule (generic function with 3 methods)

# Examples

In [8]:
function convert_to_readable(schedule, mapping)
    for (i,semester_schedule) in enumerate(schedule)
        for (j,semester_schedule) in enumerate(semester_schedule)
            schedule[i][j] = mapping[schedule[i][j]]
        end
    end
    return schedule
end

convert_to_readable (generic function with 1 method)

In [9]:
# output basic plan
cs, cs_mapping = load_major("reqs/CS.reqs")
schedule = generate_schedule(cs, 8)
output_plan(schedule, "semester_plans/example1.plan", cs_mapping)
convert_to_readable(schedule, cs_mappings)

LoadError: UndefVarError: output_plan not defined

In [10]:
# output plan considering the taken classes
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, new_mapping = load_major("reqs/CS.reqs", taken_classes)
new_schedule = generate_schedule(new_cs, 8)
convert_to_readable(new_schedule, new_mapping)

8-element Vector{Any}:
 Any["MAC2312", "Interdisciplinary-1", "COP3503", "PHY2048/PHY2048L"]
 Any["COT3100", "MAC2313", "Interdisciplinary-2", "MAS3114"]
 Any["PHY2049/PHY2049L", "COP3530", "Interdisciplinary-3", "CDA3101"]
 Any["CEN3031", "COP4020", "Interdisciplinary-4", "CIS4930-4", "CIS4930-5"]
 Any["COP4600", "CIS4301", "CIS4930-3", "COP4533", "CIS4930-2"]
 Any["Interdisciplinary-5", "CIS4930-1", "CNT4007", "Humanities2"]
 Any["Social/Behavioral2", "Humanities1", "HumOrSocOrPhy1", "EGS4034"]
 Any["STA3032", "ENC3246", "Social/Behavioral1"]

In [11]:
# output plan with more classes taken
taken_classes = Set{String}(["MAC2311", "MAC2312", "MAC2313", "COP3502", "COP3503", "EGS4034", "STA3032", "PHY2048/PHY2048L"])
new_cs, new_mapping = load_major("reqs/CS.reqs", taken_classes)
new_schedule = generate_schedule(new_cs, 6)
convert_to_readable(new_schedule, new_mapping)

6-element Vector{Any}:
 Any["COT3100", "Interdisciplinary-1", "PHY2049/PHY2049L", "Social/Behavioral2", "Humanities1"]
 Any["CDA3101", "COP3530", "Interdisciplinary-2", "Humanities2"]
 Any["CIS4301", "CEN3031", "COP4600", "CIS4930-5", "COP4533"]
 Any["CIS4930-4", "CIS4930-1", "CIS4930-3", "COP4020", "Interdisciplinary-3"]
 Any["CIS4930-2", "CNT4007", "Interdisciplinary-4", "ENC3246"]
 Any["Interdisciplinary-5", "Social/Behavioral1", "HumOrSocOrPhy1", "MAS3114"]

In [12]:
struct Monotone_map
    domain::Preorder
    codomain::Preorder
    mapping::FinFunction
    function Monotone_map(domain::Preorder, cod::Preorder, mapping::FinFunction)
      ((dom(mapping) == domain.carrier) && (codom(mapping) == cod.carrier)
      ) || error("mapping mismatch")
      return new(domain,cod,mapping)
    end
  end


function is_monotone(mm::Monotone_map)::Bool
    for comp in mm.domain.relation
        mapL = mm.mapping(comp.first)
        mapR = mm.mapping(comp.second)
        mono=false
        
        if(comp.first == comp.second) #reflexive relation
          continue
        end
        if(mapL == mapR) #pre req and class taken in same semeester
          return false;
        end

        for comparison in mm.codomain.relation

            if(mapL == comparison.first && mapR == comparison.second)
                mono=true
            end
        end
        if(!mono)
            return false
        end
    end
    return true
end

is_monotone (generic function with 1 method)

In [13]:
#check that the generated schedule creates a monotone map onto 8 semesters
cs, cs_mapping = load_major("reqs/CS.reqs")
schedule = generate_schedule(cs, 8)

semester_preorder = Preorder(FinSet(8), [1=>2, 2=>3, 3=>4, 4=>5, 5=>6, 6=>7, 7=>8])

indices = Vector{Int}(undef,length(cs.carrier))
for (index,semester) in enumerate(schedule)
    for class in semester
        indices[class] = index
    end
end




f = FinFunction(indices, cs.carrier, semester_preorder.carrier)
schedule_map = Monotone_map(cs, semester_preorder, f)

Monotone_map(Preorder(FinSet(35), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 9, 1 => 10, 1 => 11  …  28 => 29, 28 => 30, 29 => 29, 29 => 30, 30 => 30, 31 => 31, 32 => 32, 33 => 33, 34 => 34, 35 => 35],
Base.Order.ForwardOrdering())), Preorder(FinSet(8), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 4, 1 => 5, 1 => 6, 1 => 7, 1 => 8, 2 => 2, 2 => 3  …  5 => 5, 5 => 6, 5 => 7, 5 => 8, 6 => 6, 6 => 7, 6 => 8, 7 => 7, 7 => 8, 8 => 8],
Base.Order.ForwardOrdering())), FinFunction([1, 2, 3, 3, 2, 2, 4, 8, 2, 3, 4, 5, 5, 8, 6, 6, 6, 7, 7, 7, 6, 7, 6, 7, 4, 1, 2, 3, 4, 5, 5, 8, 1, 1, 8], 35, 8))

In [14]:
# check that the generated schedule specifies a monotone map between the req preorder and the semester preorder
is_monotone(schedule_map)

true

In [15]:
courses = Preorder(FinSet(4), [1=>2, 2=>3, 2=>4])
semesters = Preorder(FinSet(3), [1=>2, 2=>3])

is_monotone(Monotone_map(courses, semesters, FinFunction([1,2,3,3], FinSet(4), FinSet(3))))

true

INSERT IMAGE FROM SLIDE 10 HERE

In [16]:
is_monotone(Monotone_map(courses, semesters, FinFunction([2,1,3,3], FinSet(4), FinSet(3))))

false

INSERT IMAGE FROM SLIDE 11 HERE

In [17]:
is_monotone(Monotone_map(courses, semesters, FinFunction([1,1,2,3], FinSet(4), FinSet(3))))

false

INSERT IMAGE FROM SLIDE 12 HERE

In [18]:
taken_classes = Set{String}(["MAC2311", "COP3502"])
new_cs, mapping = load_major("reqs/CS.reqs", taken_classes)

(Preorder(FinSet(33), SortedSet([1 => 1, 1 => 2, 1 => 3, 1 => 6, 1 => 10, 1 => 12, 1 => 14, 1 => 15, 1 => 16, 1 => 17  …  26 => 27, 26 => 28, 27 => 27, 27 => 28, 28 => 28, 29 => 29, 30 => 30, 31 => 31, 32 => 32, 33 => 33],
Base.Order.ForwardOrdering())), Dict{Any, Any}(5 => "PHY2048/PHY2048L", 16 => "COP4533", 20 => "CIS4930-3", 12 => "CEN3031", 24 => "Interdisciplinary-1", 28 => "Interdisciplinary-5", 8 => "COP3503", 17 => "CNT4007", 30 => "Humanities2", 1 => "MAC2312"…))

# Why this works

### First we show why we can take from the "bottom".

So what we mean by classes on the "bottom" (with all prerequisites fulfilled) is actually minimal elements (with no elements below them). 




![title](Images/MinmalElement.png)

(from https://en.wikipedia.org/wiki/Maximal_and_minimal_elements)


## Definition

![title](Images/SkeletalDefinition.png)

The key property to note is that our preorder is skeletal because no two classes can be prerequisites of each other. This along with finiteness guarantees we can simply take minimal elements to create a full schedule.

### Proposition
      Every non-minimal object of a finite skeletal preorder is above a minimal element. 

### Proof

We proceed by showing that a skeletal preorder with a non-minimal object $a_{0}$ that is not above a minimal element is infinite.

Let (P, $\leq$), be a skeletal preorder. Suppose $a_{0}$ is above a minimal element. This implies that there exists an $a_{1}$ with  $ a_{1} < a_{0} $. Clearly $a_{1}$ cannot be above a minimal element either. Now for every positive integer n, if $a_{n}$ is not above a minimal element,  we can take $ a_{n+1} < a_{n} $ and where $a_{n+1}$ is not above a minimal element. Each $a_{n}$ is distinct because our preorder is skeletal so we have an infinite sequence of unique preorder elements $a_{n}$.

### Proposition
    A preorder with a minimal element removed is still a preorder and skeletality preserved

### Proof idea
Removing an element does not affect transitivity. $x \leq x$ still holds for all remaining elements.


Similarly, skeletality is not affected by removing an element.

For transitivity, if a,b,c are elements that have not been removed,
if $a \leq b$ and $b \leq c$ then $a \leq c$ has not been removed from our smaller preorder either.

### Corollary
There are always minimal elements to remove from a finite skeletal preorder and doing so yields a skeletal preorder that is one element smaller. This means that the method of removing minimal elements will exhaust the preorder of classes with enough steps. Now that it is has been shown that we can build from bottom up, we show that doing so does indeed preserve order.

## Conclusion

We try to 4-5 minimal elements with each semester to represent taking 12-15 credits in a semester in our implementation but in general:

### 1. A valid schedule determines a strict monotone map
### 2.  A strict monotone map to a linear order uniquely determines a valid schedule

Let $\underline{n}$ stand for the linear order of the first n elements of the natural numbers.

### Proposition

Let S be a valid schedule with n semesters. Let (P, $\leq$) be a class preorder. Let $f: P \rightarrow \underline{n}$ be defined by $f(a)$ = the semester a is taken. Then f is a strict monotone map.

### Proof

Suppose $a<b$. Then because S is valid, $b$ is not minimal until $a$ is taken. So if $a$ is taken in semester $k$, then $b$ is valid starting semester $k+1$ at the earliest. Thus, $f(b) \geq k+1 > k = f(a)$

### Proposition

Let $f:P \rightarrow \underline{n}$ be a strict monotone map. Then the schedule S with semester k consisting of $f^{-1}(k)$ is valid.

### Proof
We attempt to produce S by choosing $f^{-1}(k)$ for our semester at each step.

Suppose S is not valid. Then we must have some semester k for which a class $b$ is taken but not valid. That is, $b$ is taken but not yet made minimal. This means that some class $a$ with $a<b$ exists in the preorder when semester k was chosen. This means that $a$ was chosen on semester k or later. Thus, $f(a) \geq k$. But then $a<b$ and $f(b) = k \leq f(a)$, violating strictness.

What this means is that we can perfectly think of preorders as prerequisites and strict monotone maps to total orders as semester schedules. 

## We get a few corollaries:
1. It is possible to graduate in n semester (administration allowing) semester if there exists a monotone map from the preorder of remaining classes to a total order of size n.

2. A schedule is possible if it represents a strict monotone map

3. It is possible to graduate taking in $k$ semesters taking $n_{i}$ classes on semester i, if there exists a strict monotone map $f$ to a total order of size $k$ with |$f^{-1}$(x) = $n_{x}$ |

# A generalization

Our method of taking the schedule from bottom up easily generalizes to adding in prerequisites that include the logical **or** on top of **and**. Using the following formula, we can write any set of prerequisites with all instances of **and** coming before any instance of **or**.

$a \wedge (b \lor c) = (a \lor b) \wedge (a \lor c)$

With for instance of $a \lor b$, we create a new node with $a,b \leq (a \lor b)$
When class a or class b is taken and flagged for removal, we remove $a \lor b$ from the preorder as well. With this change, all prerequisites are removed as they are fulfilled so the minimal elements of our graph are valid to be taken.
